In [2]:
import sys
sys.path.append('/Users/fdjim/Desktop/PDS_CODE/src_acw_calculation_stats')

from data_loader import load_fmri_data, load_regressors
from roi_creation import fetch_aal_atlas, create_label_to_pseudo_index, get_pseudo_index, create_masker, extract_timeseries, extract_timeseries_for_roi
from region_lists import auditory_cortex_regions, visual_cortex_regions, motor_cortex_regions, dmn_regions, fpn_regions, salience_network_regions, limbic_system_regions
import pandas as pd
import numpy as np


In [3]:
# Load empricial fMRI data and regressors
data_dir = '/Users/fdjim/Desktop/PDS_FULL/derivatives/fmriprep'
subjects = [
    "sub-003P", "sub-004P", "sub-005C", "sub-005P", "sub-006C",
    "sub-007C", "sub-007P", "sub-008P", "sub-009C", "sub-010P",
    "sub-013C", "sub-014C", "sub-014P", "sub-015C", "sub-015P",
    "sub-016C", "sub-016P", "sub-017C", "sub-018C", "sub-018P",
    "sub-019C", "sub-019P", "sub-020C", "sub-020P", "sub-021C",
    "sub-023P", "sub-024C", "sub-024P", "sub-025C", "sub-026C",
    "sub-027C", "sub-028C", "sub-029C", "sub-030C", "sub-030P",
    "sub-031C", "sub-031P", "sub-032P", "sub-033P", "sub-035P",
    "sub-037P", "sub-039P", "sub-041P", "sub-042P", "sub-043P",
    "sub-045P", "sub-046P", "sub-047P", "sub-048P", "sub-049P",
    "sub-050P", "sub-051P", "sub-052P", "sub-055P", "sub-056P",
    "sub-061P", "sub-066P", "sub-067P", "sub-069P", "sub-071P",
    "sub-075P"
]


In [4]:
# Split the subjects into groups for the analysis
diagnosis_df = pd.read_csv('/Users/fdjim/Desktop/PDS_FULL/participants.tsv', sep='\t')
# Create a dictionary mapping subject IDs to diagnosis
diagnosis_dict = dict(zip(diagnosis_df['participant_id'], diagnosis_df['dx']))
# Split subjects into groups based on diagnosis
subjects_hc = []
subjects_mdd = []
subjects_sz = []
for subject in subjects:
    if subject in diagnosis_dict:
        if diagnosis_dict[subject] == 'hc':
            subjects_hc.append(subject)
        elif diagnosis_dict[subject] == 'mdd':
            subjects_mdd.append(subject)
        elif diagnosis_dict[subject] == 'sz':
            subjects_sz.append(subject)
    else:
        print(f"Warning: Subject {subject} not found in diagnosis file")
print(f"HC: {subjects_hc}")
print(f"MDD: {subjects_mdd}")
print(f"SZ: {subjects_sz}")


HC: ['sub-005C', 'sub-006C', 'sub-007C', 'sub-009C', 'sub-013C', 'sub-014C', 'sub-015C', 'sub-016C', 'sub-017C', 'sub-018C', 'sub-019C', 'sub-020C', 'sub-021C', 'sub-024C', 'sub-025C', 'sub-026C', 'sub-027C', 'sub-028C', 'sub-029C', 'sub-030C', 'sub-031C']
MDD: ['sub-003P', 'sub-004P', 'sub-005P', 'sub-007P', 'sub-008P', 'sub-010P', 'sub-014P', 'sub-020P', 'sub-031P', 'sub-032P', 'sub-033P', 'sub-035P', 'sub-037P', 'sub-042P', 'sub-046P', 'sub-048P', 'sub-051P', 'sub-052P', 'sub-066P', 'sub-067P', 'sub-075P']
SZ: ['sub-015P', 'sub-016P', 'sub-018P', 'sub-019P', 'sub-023P', 'sub-024P', 'sub-030P', 'sub-039P', 'sub-041P', 'sub-043P', 'sub-045P', 'sub-047P', 'sub-049P', 'sub-050P', 'sub-055P', 'sub-056P', 'sub-061P', 'sub-069P', 'sub-071P']


In [5]:
# Fetch atlas and create label to pseudo-index mapping
labels_aal, indices_aal, aal_img = fetch_aal_atlas()
pseudo_idx_aal = create_label_to_pseudo_index(labels_aal)
pseudo_idx_auditory = get_pseudo_index(pseudo_idx_aal, auditory_cortex_regions)
pseudo_idx_visual = get_pseudo_index(pseudo_idx_aal, visual_cortex_regions)
pseudo_idx_motor = get_pseudo_index(pseudo_idx_aal, motor_cortex_regions)
pseudo_idx_dmn = get_pseudo_index(pseudo_idx_aal, dmn_regions)
pseudo_idx_fpn = get_pseudo_index(pseudo_idx_aal, fpn_regions)
pseudo_idx_salience = get_pseudo_index(pseudo_idx_aal, salience_network_regions)
pseudo_idx_limbic = get_pseudo_index(pseudo_idx_aal, limbic_system_regions)

pseudo_indices = [pseudo_idx_auditory, pseudo_idx_visual, pseudo_idx_motor, pseudo_idx_dmn, pseudo_idx_fpn, pseudo_idx_salience, pseudo_idx_limbic]

In [18]:
print(indices_aal)
print(labels_aal)

['2001', '2002', '2101', '2102', '2111', '2112', '2201', '2202', '2211', '2212', '2301', '2302', '2311', '2312', '2321', '2322', '2331', '2332', '2401', '2402', '2501', '2502', '2601', '2602', '2611', '2612', '2701', '2702', '3001', '3002', '4001', '4002', '4011', '4012', '4021', '4022', '4101', '4102', '4111', '4112', '4201', '4202', '5001', '5002', '5011', '5012', '5021', '5022', '5101', '5102', '5201', '5202', '5301', '5302', '5401', '5402', '6001', '6002', '6101', '6102', '6201', '6202', '6211', '6212', '6221', '6222', '6301', '6302', '6401', '6402', '7001', '7002', '7011', '7012', '7021', '7022', '7101', '7102', '8101', '8102', '8111', '8112', '8121', '8122', '8201', '8202', '8211', '8212', '8301', '8302', '9001', '9002', '9011', '9012', '9021', '9022', '9031', '9032', '9041', '9042', '9051', '9052', '9061', '9062', '9071', '9072', '9081', '9082', '9100', '9110', '9120', '9130', '9140', '9150', '9160', '9170']
['Precentral_L', 'Precentral_R', 'Frontal_Sup_L', 'Frontal_Sup_R', 'Fro

In [11]:
# Create a reverse mapping from index to region name
idx_to_region = {v: k for k, v in pseudo_idx_aal.items()}

# Function to map indices to region names
def map_indices_to_regions(indices):
    return [idx_to_region[idx] for idx in indices]

# Map each network's indices to region names
network_regions = {
    'Auditory': map_indices_to_regions(pseudo_idx_auditory),
    'Visual': map_indices_to_regions(pseudo_idx_visual),
    'Motor': map_indices_to_regions(pseudo_idx_motor),
    'DMN': map_indices_to_regions(pseudo_idx_dmn),
    'FPN': map_indices_to_regions(pseudo_idx_fpn),
    'Salience': map_indices_to_regions(pseudo_idx_salience),
    'Limbic': map_indices_to_regions(pseudo_idx_limbic)
}

# Print the regions for each network
for network, regions in network_regions.items():
    print(f"\n{network} Network Regions:")
    for region in regions:
        print(f"- {region}")


Auditory Network Regions:
- Heschl_L
- Heschl_R
- Temporal_Sup_L
- Temporal_Sup_R

Visual Network Regions:
- Calcarine_L
- Calcarine_R
- Cuneus_L
- Cuneus_R
- Lingual_L
- Lingual_R
- Occipital_Sup_L
- Occipital_Sup_R
- Occipital_Mid_L
- Occipital_Mid_R

Motor Network Regions:
- Precentral_L
- Precentral_R
- Supp_Motor_Area_L
- Supp_Motor_Area_R

DMN Network Regions:
- Frontal_Sup_Medial_L
- Frontal_Sup_Medial_R
- Cingulum_Post_L
- Cingulum_Post_R
- Angular_L
- Angular_R

FPN Network Regions:
- Frontal_Mid_L
- Frontal_Mid_R
- Parietal_Inf_L
- Parietal_Inf_R

Salience Network Regions:
- Insula_L
- Insula_R
- Cingulum_Ant_L
- Cingulum_Ant_R

Limbic Network Regions:
- Amygdala_L
- Amygdala_R
- Hippocampus_L
- Hippocampus_R


In [8]:
# Create masker object, bandpass filter and smooth the data
smoothing_fwhm = 2
high_pass = 0.008
low_pass = 0.24
t_r = 2

masker = create_masker(aal_img, smoothing_fwhm=smoothing_fwhm, high_pass=high_pass, low_pass=low_pass, 
                       t_r=t_r, standardize=True)

Masker object created.


### First Extract & Save HC timeseries

In [9]:
ts_hc = []
task = 'pds'

for group in [subjects_hc]:
    fmri_imgs = load_fmri_data(data_dir, group, task)
    regressors = load_regressors(data_dir, group, task)
    # Extract time series data
    ts_group = extract_timeseries(masker, fmri_imgs, regressors)
    if group == subjects_hc:
        ts_hc.extend(ts_group)

Shape of fMRI data for subject sub-005C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-006C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-007C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-009C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-013C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-014C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-015C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-016C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-017C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-018C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-019C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-020C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-021C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-024C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-025C : (97, 115, 97, 312)
Shape of fMRI data for subject sub-026C : (97, 115, 97, 312)
Shape of fMRI data for s

In [34]:
# Creating a dictionary to store the timeseries for each subject and region

ts_hc_dict = {}
for i in range(len(subjects_hc)):
    subject = subjects_hc[i]
    auditory_timeseries = ts_hc[i][:, pseudo_idx_auditory]
    visual_timeseries = ts_hc[i][:, pseudo_idx_visual]
    motor_timeseries = ts_hc[i][:, pseudo_idx_motor]
    dmn_timeseries = ts_hc[i][:, pseudo_idx_dmn]
    fpn_timeseries = ts_hc[i][:, pseudo_idx_fpn]
    salience_timeseries = ts_hc[i][:, pseudo_idx_salience]
    limbic_timeseries = ts_hc[i][:, pseudo_idx_limbic]
    ts_hc_dict[subject] = {
        'auditory': auditory_timeseries,
        'visual': visual_timeseries,
        'motor': motor_timeseries,
        'dmn': dmn_timeseries,
        'fpn': fpn_timeseries,
        'salience': salience_timeseries,
        'limbic': limbic_timeseries
    }


In [50]:
# Create lists to store data
subjects = []
regions = []
all_timeseries = []

# Extract data from dictionary
for subject, region_dict in ts_hc_dict.items():
    for region, ts in region_dict.items():
        # Transpose timeseries from (312, N) to (N, 312)
        ts_transposed = ts.T
        
        # For each individual timeseries in the region
        for single_ts in ts_transposed:
            subjects.append(subject)
            regions.append(region)
            all_timeseries.append(single_ts)

# Convert timeseries list to a numpy array for easier handling
all_timeseries_array = np.array(all_timeseries)

# Create column names for timepoints
timepoint_cols = [f'timepoint_{i+1}' for i in range(all_timeseries_array.shape[1])]

# Create the DataFrame all at once
df = pd.DataFrame(
    np.column_stack([subjects, regions, all_timeseries_array]),
    columns=['subject', 'region'] + timepoint_cols
)

# Save to CSV
df.to_csv('timeseries_data_wide.csv', index=False)

### Now for MDD and SZ

In [51]:
ts_mdd = []
ts_sz = []
task = 'pds'

# already extracted HC
for group in [subjects_mdd, subjects_sz]:
    fmri_imgs = load_fmri_data(data_dir, group, task)
    regressors = load_regressors(data_dir, group, task)
    # Extract time series data
    ts_group = extract_timeseries(masker, fmri_imgs, regressors)
    if group == subjects_mdd:
        ts_mdd.extend(ts_group)
    elif group == subjects_sz:
        ts_sz.extend(ts_group)


Shape of fMRI data for subject sub-003P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-004P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-005P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-007P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-008P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-010P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-014P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-020P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-031P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-032P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-033P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-035P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-037P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-042P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-046P : (97, 115, 97, 312)
Shape of fMRI data for subject sub-048P : (97, 115, 97, 312)
Shape of fMRI data for s

In [57]:
# Creating a dictionary to store the timeseries for each subject and region

ts_mdd_dict = {}
for i in range(len(subjects_mdd)):
    subject = subjects_mdd[i]
    auditory_timeseries = ts_mdd[i][:, pseudo_idx_auditory]
    visual_timeseries = ts_mdd[i][:, pseudo_idx_visual]
    motor_timeseries = ts_mdd[i][:, pseudo_idx_motor]
    dmn_timeseries = ts_mdd[i][:, pseudo_idx_dmn]
    fpn_timeseries = ts_mdd[i][:, pseudo_idx_fpn]
    salience_timeseries = ts_mdd[i][:, pseudo_idx_salience]
    limbic_timeseries = ts_mdd[i][:, pseudo_idx_limbic]
    ts_mdd_dict[subject] = {
        'auditory': auditory_timeseries,
        'visual': visual_timeseries,
        'motor': motor_timeseries,
        'dmn': dmn_timeseries,
        'fpn': fpn_timeseries,
        'salience': salience_timeseries,
        'limbic': limbic_timeseries
    }

ts_sz_dict = {}
for i in range(len(subjects_sz)):
    subject = subjects_sz[i]
    auditory_timeseries = ts_sz[i][:, pseudo_idx_auditory]
    visual_timeseries = ts_sz[i][:, pseudo_idx_visual]
    motor_timeseries = ts_sz[i][:, pseudo_idx_motor]
    dmn_timeseries = ts_sz[i][:, pseudo_idx_dmn]
    fpn_timeseries = ts_sz[i][:, pseudo_idx_fpn]
    salience_timeseries = ts_sz[i][:, pseudo_idx_salience]
    limbic_timeseries = ts_sz[i][:, pseudo_idx_limbic]
    ts_sz_dict[subject] = {
        'auditory': auditory_timeseries,
        'visual': visual_timeseries,
        'motor': motor_timeseries,
        'dmn': dmn_timeseries,
        'fpn': fpn_timeseries,
        'salience': salience_timeseries,
        'limbic': limbic_timeseries
    }


In [62]:
# Create lists to store data
subjects = []
regions = []
all_timeseries = []

# Extract data from dictionary
for subject, region_dict in ts_mdd_dict.items():
    for region, ts in region_dict.items():
        # Transpose timeseries from (312, N) to (N, 312)
        ts_transposed = ts.T
        
        # For each individual timeseries in the region
        for single_ts in ts_transposed:
            subjects.append(subject)
            regions.append(region)
            all_timeseries.append(single_ts)

# Convert timeseries list to a numpy array for easier handling
all_timeseries_array = np.array(all_timeseries)

# Create column names for timepoints
timepoint_cols = [f'timepoint_{i+1}' for i in range(all_timeseries_array.shape[1])]

# Create the DataFrame all at once
df = pd.DataFrame(
    np.column_stack([subjects, regions, all_timeseries_array]),
    columns=['subject', 'region'] + timepoint_cols
)

# Save to CSV
df.to_csv('timeseries_data_wide_mdd.csv', index=False)

In [63]:
# Create lists to store data
subjects = []
regions = []
all_timeseries = []

# Extract data from dictionary
for subject, region_dict in ts_sz_dict.items():
    for region, ts in region_dict.items():
        # Transpose timeseries from (312, N) to (N, 312)
        ts_transposed = ts.T
        
        # For each individual timeseries in the region
        for single_ts in ts_transposed:
            subjects.append(subject)
            regions.append(region)
            all_timeseries.append(single_ts)

# Convert timeseries list to a numpy array for easier handling
all_timeseries_array = np.array(all_timeseries)

# Create column names for timepoints
timepoint_cols = [f'timepoint_{i+1}' for i in range(all_timeseries_array.shape[1])]

# Create the DataFrame all at once
df = pd.DataFrame(
    np.column_stack([subjects, regions, all_timeseries_array]),
    columns=['subject', 'region'] + timepoint_cols
)

# Save to CSV
df.to_csv('timeseries_data_wide_sz.csv', index=False)